Wingate et al., AISTATS 2011, Algorithm 4
---

This paper by Wingate et al. proposes methods for recording, editing, and playing-back program traces, which become core components in PTO.

Algorithm 4 in this paper is "An illustration how the number and type of random choices can change." It is a short program which implicitly defines a distribution. 

The algorithm is (in Matlab):

```
1: m=poissrnd();
2: for i=1:m
3:     X(i) = gammarnd();
4: end;
5: for i=m+1:2*m
6:     X(i) = randn;
7: end;
```

It's intended as a distribution we might want to sample from, but for PTO purposes, let's define a fitness function on it -- `sum` -- and try to optimise it. The reason it's interesting for both Wingate et al. and for us is that the number `m` can change, which, in a naive implementation of program traces, would mean we were interpreting old trace material, previously used to sample from a gamma distribution, to sample from a normal, which would lead to bad results. Wingate et al. propose a better program trace implementation which avoids that, and which we therefore adopt in PTO.


Implementing the problem for PTO
---

We first define the problem by defining the generator and fitness. Numpy provides the functions we need, but we are not using Numpy in our PTO generator code, for now (we will still use it for analysis, though). Python's standard library `random` module provides `gammavariate` and `normalvariate`, but it doesn't provide a Poisson variate, so we will implement that first.

In [1]:
import random, math
def poissonvariate(lamb):
    """A method which will be slow for large lambda
    From https://www.johndcook.com/blog/2010/06/14/generating-poisson-random-values/
init:
         Let L ← exp(−λ), k ← 0 and p ← 1.
do:
         k ← k + 1.
         Generate uniform random number u in [0,1] and let p ← p × u.
while p > L.
return k − 1.

>>> sum([poissonvariate(2.5) for _ in range(100)]) / 100.0
 -> a value near 2.5
    """
    L = math.exp(-lamb)
    k = 0
    p = 1
    while p > L:
        k += 1
        u = random.random()
        p *= u
    return k - 1

Then, the generator closely mimics the original Matlab. Fitness is just `sum` as stated.

In [2]:
def generator():
    m = poissonvariate(10)
    X = []
    for i in range(m):
        X.append(random.gammavariate(1, 1)) # alpha=1, beta=1
    for i in range(m):
        X.append(random.normalvariate(0, 1))
    return X
fitness = sum

In [3]:
from PTO import random, random_function, solve

In [4]:
ind, fit = solve(generator, fitness, solver="HC", effort=1.0)
print(ind)
print("fitness: ", fit)

[0.08358466201190759, 0.48143161905919946, 0.23853223262173445, 0.6421584913865963, 1.9571024812345295, 1.003821634311713, 0.20572271768498082, 4.417013897601947, 1.1649710388394199, 3.40100922232921, 1.0169776821846943, 0.060491490083636655, 1.9445634926319095, 0.376549334988143, 0.04109482285905346, 1.6239008268679758, 0.06666455317723373, 1.787518507864078, -2.141203655902093, -0.04143280744123887, -0.01911717380521149, 2.087048987735042, 2.0413363142739085, 1.4356382390520834, -0.8600251825586029, 0.49526039305865066, -0.4596633240588841, 0.06607474601206834, 1.872817007804373, 0.6173207547654004, 0.09706202473336747, 0.17148841631155817, 3.614169590582924, -0.8771742944702844]
('fitness: ', 28.61270874383103)


Experiments and Analysis
---

Next, we can do a large run and carry out some analysis on our results. 
We'll import functions provided by PTO, and also use scipy.


In [5]:
from PTO import compare_all, stat_summary
import scipy.stats # only for post-run analysis

In [13]:
# compare_all: you can pass in sizes if needed, else None. It uses 'budget', not 'effort'.
results = compare_all(fitness, [generator], sizes=None, 
                      methods=["HC", "LA", "EA"], str_traces=[False, True], 
                      budget=10000, num_runs=10)

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


**Experiment 1**: Compare structured trace with linear, using hill-climbing: structured trace wins by far, of course.

In [14]:
d0, d1 = results[(None, False, 'generator', 'HC')], results[(None, True, 'generator', 'HC')]
print(stat_summary(d0))
print(stat_summary(d1))
print(scipy.stats.ttest_ind(d0, d1))

mean 124.37 std 24.10 min 84.70 med 120.76 max 175.12
mean 344.63 std 26.21 min 305.41 med 336.87 max 391.70
Ttest_indResult(statistic=-18.559297810658265, pvalue=3.4870167422353794e-13)


**Experiment 2**: Compare hill-climbing with EA, using structured trace: HC wins!

In [15]:
d0, d1 = results[(None, True, 'generator', 'HC')], results[(None, True, 'generator', 'EA')]
print(stat_summary(d0))
print(stat_summary(d1))
print(scipy.stats.ttest_ind(d0, d1))

mean 344.63 std 26.21 min 305.41 med 336.87 max 391.70
mean 283.92 std 10.81 min 267.45 med 284.41 max 299.74
Ttest_indResult(statistic=6.4230721300762541, pvalue=4.7942300292880871e-06)


**Experiment 3**: Compare HC with LAHC, using structured trace: HC wins! Stupid algorithms are good with the right representation?!

In [17]:
d0, d1 = results[(None, True, 'generator', 'HC')], results[(None, True, 'generator', 'LA')]
print(stat_summary(d0))
print(stat_summary(d1))
print(scipy.stats.ttest_ind(d0, d1))

mean 344.63 std 26.21 min 305.41 med 336.87 max 391.70
mean 299.75 std 18.42 min 274.29 med 299.51 max 330.42
Ttest_indResult(statistic=4.202115967932464, pvalue=0.00053569475594783056)


Conclusions and future work
---

We have demonstrated the use of variable-length and heterogeneously-typed (ie different distributions) program traces, and how to carry out an experiment using PTO's provided tools, leading to some simple experiment results.

Some ideas to test in future:

* Try an alternative generator which interleaves the gamma and normal sampling.

* Instead of setting `m` at the start, decide "whether to stop" at each
iteration -- need to make sure `m` distribution is still Poisson, though.
